# Bodhiac Monthly Data Refresh - ETL

This script contains the monthly data refresh of Bodhiac BV.
It was designed by Valiant Quants to automatically add new data and calculations to their BI dashboards.

## Libraries

In [59]:
import os
import io
import requests
import pandas as pd
import datetime
import pytz
import csv
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from urllib.request import urlretrieve

## Config

IMPORTANT: set the correct files to refresh, path location and Google Drive API linked to your private key.

In [61]:
# set the path where you want to save the file
save_path = r"C:\Users\Vadim\data-sources\bodhiac" # Local data folder for temp storage

# set the name you want to give the file
file_name = r'bodhiac-transactions.csv' # Invoices

# set the file ID of the file you want to download
file_id = r'1zT9AWJoBClDMqGWeTqR3sgAH80U9Ztrs0EkGNXH2sVk' # r'18UVzEkRwmj__NWNPvucRzDq-OkWHK4MqdSvpMJgle6o'

# Set the ID of the folder that you want to download files from
folder_id = '1o0GOIrPh6VlE0_nHGVBQm-VnPiB30Egj' # DATA / INPUT

# Set the ID of the spreadsheet that you want to update
spreadsheet_id = r'1zT9AWJoBClDMqGWeTqR3sgAH80U9Ztrs0EkGNXH2sVk'  # DASHBOARD FILE (IF DIFFERENT FROM ABOVE)

# Set the range of cells that you want to update
range_name = 'BANK STATEMENTS!A1:ZZ10000'

# Set the range of cells that you want to update
range_transactions = 'E:E'


In [49]:
# set the credentials to authenticate with the Google Drive API
creds = service_account.Credentials.from_service_account_file(
    r'C:\Users\Vadim\data-config\Google\valiantquants-7886443d4391.json')

# set the MIME type of the export format you want
mime_type = r'text/csv'

#print(os.path.join(save_path, file_name))

# ETL Extraction

## Get Invoices Data from Dashboard file

Below we will export the existing data from the file that we are using in the Looker dashboard and store it locally.

In [36]:
try:
    # create a Google Drive API client
    service = build('drive', 'v3', credentials=creds)

    # export the file contents
    request = service.files().export(fileId=file_id, mimeType=mime_type)
    file_content = io.BytesIO()
    downloader = MediaIoBaseDownload(file_content, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print(f'Download {int(status.progress() * 100)}.')
    
    # save the file to disk
    with open(os.path.join(save_path, file_name), 'wb') as f:
        f.write(file_content.getbuffer())

    print(f'Download complete: {os.path.join(save_path, file_name)}')

except HttpError as error:
    print(f'An error occurred: {error}')

"\ntry:\n    # create a Google Drive API client\n    service = build('drive', 'v3', credentials=creds)\n\n    # export the file contents\n    request = service.files().export(fileId=file_id, mimeType=mime_type)\n    file_content = io.BytesIO()\n    downloader = MediaIoBaseDownload(file_content, request)\n    done = False\n    while done is False:\n        status, done = downloader.next_chunk()\n        print(f'Download {int(status.progress() * 100)}.')\n    \n    # save the file to disk\n    with open(os.path.join(save_path, file_name), 'wb') as f:\n        f.write(file_content.getbuffer())\n\n    print(f'Download complete: {os.path.join(save_path, file_name)}')\n\nexcept HttpError as error:\n    print(f'An error occurred: {error}')\n"

## Download files from INPUT sftp folder

New files to be processed.

In [34]:
# List all files in the folder
service = build('drive', 'v3', credentials=creds)
query = "parents='" + folder_id + "' and trashed=false"
results = service.files().list(q=query, fields="nextPageToken, files(id, name, mimeType)").execute()
files = results.get('files', [])

# Download each file
if not files:
    print('No files found.')
else:
    for file in files:
        try:
            file_id = file['id']
            file_name = file['name']
            file_mime_type = file['mimeType']
            if file_mime_type != 'application/vnd.google-apps.folder':  
                # export the file contents
                request = service.files().get_media(fileId=file_id)
                file_content = io.BytesIO()
                downloader = MediaIoBaseDownload(file_content, request)
                done = False
                while done is False:
                    status, done = downloader.next_chunk()
                    print(f'Download {int(status.progress() * 100)}.')
                # save the file to disk
                with open(os.path.join(save_path, file_name), 'wb') as f:
                    f.write(file_content.getbuffer())
                print(f'Download complete: {os.path.join(save_path, file_name)}')
        except HttpError as error:
            print('An error occurred: %s' % error)


Download 100.
Download complete: C:\Users\Vadim\data-showcase\data\bodhiac-invoices-full.csv


# ETL Transformations

In [46]:
# Replace 'path/to/csv/file.csv' with the actual path to your CSV file
df = pd.read_csv(os.path.join(save_path, file_name))

# Display the first few rows of the DataFrame
print(df.head())

        ID                   Gateway ID                       Email  \
0  4909439  ch_1HoSNgBte6Tcs36KIUaIl77Y         info@moonbeetle.com   
1  4945862                          NaN           hello@bodhiac.com   
2  5060382                          NaN  gitte.verstricht@gmail.com   
3  5060383                          NaN     juliewindey@hotmail.com   
4  5060385                          NaN    joellesimons@hotmail.com   

                 Name  User ID                                    Product  \
0    Joris Heyndrickx  4759007                           Beginners Series   
1  Alice from Bodhiac  3967004                     Weekly BODHIAC Program   
2    Gitte Verstricht  4871074  Yearly Membership (Incl 7-day free trial)   
3        Julie Windey  4871075  Yearly Membership (Incl 7-day free trial)   
4       Simons joëlle  4871072  Yearly Membership (Incl 7-day free trial)   

   Product ID Coupon Total Discount  ...               Last Attempt  \
0       42533   TEST     1        0  ..

C:\Users\Vadim\data-showcase\data\bodhiac-invoices-full.csv


# ETL Load

In [91]:
# Clear the data in the sheet
service = build('sheets', 'v4', credentials=creds)
clear_values_request_body = {}
service.spreadsheets().values().clear(spreadsheetId=spreadsheet_id, range=range_name, body=clear_values_request_body).execute()

{'spreadsheetId': '1zT9AWJoBClDMqGWeTqR3sgAH80U9Ztrs0EkGNXH2sVk',
 'clearedRange': "'BANK STATEMENTS'!A1:AE1979"}

In [92]:
# Write new data to the sheet
with open(os.path.join(save_path, file_name), 'r') as csv_file:
    reader = csv.reader(csv_file)
    values = list(reader)
    body = {
        'values': values
    }
    write_values_request_body = {
        'valueInputOption': 'USER_ENTERED',
        'data': [
            {
                'range': range_name,
                'majorDimension': 'ROWS',
                'values': values
            }
        ]
    }
    service.spreadsheets().values().batchUpdate(spreadsheetId=spreadsheet_id, body=write_values_request_body).execute()

print('Data updated successfully.')

Data updated successfully.


In [50]:
print(os.path.join(save_path, file_name))

C:\Users\Vadim\data-showcase\data\bodhiac-transactions.csv


In [93]:
# Use the Google Sheets API to read the data from the spreadsheet and filter it based on the date criteria.
sheet_values = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_transactions).execute().get('values', [])
sheet_metadata = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
sheet_id = sheet_metadata['sheets'][0]['properties']['sheetId'])
sheet_values = sheet_values[1:]

filtered_values = [row for row in sheet_values if row and datetime.datetime.strptime(row[0], "%d/%m/%Y").date() >= datetime.date(2022, 3, 1)]

print(filtered_values)

[['01/03/2022'], ['01/03/2022'], ['01/03/2022'], ['01/03/2022'], ['03/03/2022'], ['03/03/2022'], ['04/03/2022'], ['08/03/2022'], ['09/03/2022'], ['10/03/2022'], ['11/03/2022'], ['11/03/2022'], ['14/03/2022'], ['16/03/2022'], ['16/03/2022'], ['17/03/2022'], ['18/03/2022'], ['24/03/2022'], ['25/03/2022'], ['28/03/2022'], ['29/03/2022'], ['29/03/2022'], ['29/03/2022'], ['31/03/2022'], ['01/04/2022'], ['04/04/2022'], ['06/04/2022'], ['06/04/2022'], ['07/04/2022'], ['08/04/2022'], ['13/04/2022'], ['14/04/2022'], ['19/04/2022'], ['19/04/2022'], ['20/04/2022'], ['20/04/2022'], ['21/04/2022'], ['22/04/2022'], ['25/04/2022'], ['26/04/2022'], ['26/04/2022'], ['26/04/2022'], ['27/04/2022'], ['27/04/2022'], ['28/04/2022'], ['29/04/2022'], ['02/05/2022'], ['03/05/2022'], ['05/05/2022'], ['06/05/2022'], ['10/05/2022'], ['10/05/2022'], ['10/05/2022'], ['11/05/2022'], ['11/05/2022'], ['12/05/2022'], ['12/05/2022'], ['16/05/2022'], ['17/05/2022'], ['18/05/2022'], ['19/05/2022'], ['19/05/2022'], ['23/05

In [94]:
# Use the Google Sheets API to delete the filtered data from the spreadsheet.
if filtered_values:
    num_rows = len(filtered_values)
    body = {'requests': [{'deleteRange': {'range': {'sheetId': sheet_id, 'startRowIndex': 1, 'endRowIndex': num_rows+1, 'startColumnIndex': 0, 'endColumnIndex': 100}, 'shiftDimension': 'ROWS'}}]}
    service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body=body).execute()
    print(f'{num_rows} rows deleted.')
else:
    print('No rows to delete.')

128 rows deleted.


In [87]:
service.spreadsheets().sheets().get(spreadsheetId=spreadsheet_id, range=range_transactions)

AttributeError: 'Resource' object has no attribute 'get'

In [104]:

# Call the spreadsheets().get() method to get the sheet ID
sheet_metadata = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
print(sheet_metadata['sheets'][0]['properties']['sheetId'])

1225125937
